In [140]:
import pandas as pd
import ast 
import numpy as np
import csv

In [141]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

In [142]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        for key, value in cards_dict.items():
            if value.startswith("Carte et informations d'accès"):
                reordered_cards['card_1'] = value.split('\n') 
            elif value.startswith("Présentation"):
                reordered_cards['card_2'] = value.split('\n') 
            elif value.startswith("Horaires et coordonnées"):
                reordered_cards['card_3'] = value.split('\n') 
            elif value.startswith("Tarifs"):
                reordered_cards['card_4'] = value.split('\n') 
            elif value.startswith("Informations légales"):
                reordered_cards['card_5'] = value.split('\n') 
        
        return reordered_cards
    except:
        return {
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)

In [143]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [144]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [145]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [146]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)

data_combined.drop(columns = 'card_5', inplace = True)


In [147]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else np.nan)

In [148]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Site web']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [149]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [150]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [151]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [152]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [153]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [154]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [155]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [156]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [157]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [158]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [159]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [160]:
data_combined.drop(columns=['URL','card_1','card_2'],inplace=True)
data_combined.columns

Index(['Doctor Name', 'Specialty', 'card_3', 'card_4', 'Latitude', 'Longitude',
       'Skills Processed', 'RPPS', 'Nb Skills', 'Présentation',
       'Langues parlées', 'Diplômes nationaux et universitaires',
       'Autres formations', 'Expériences', 'Site web',
       'Travaux et publications', 'Prix et distinctions', 'site', 'Formations',
       'Diplômes nationaux et universitaires b', 'Nb caractères présentation',
       'Autres formations b', 'Nb langues', 'Expériences b', 'Carac_card_1',
       'Transport', 'Parking', 'Infos', 'Coordonnées', 'Sans RDV'],
      dtype='object')

Ouverture annuaire santé

In [137]:
annuaire = pd.read_csv('data/Annuaire_Sante.csv')
annuaire.head()

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_70643/3626557920.py:1: DtypeWarning: Columns (24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/Annuaire_Sante.csv')


,typedidentifiantpp,identifiantpp,identificationnationalepp,codecivilité,codeprofession,libelléprofession,codemodeexercice,libellémodeexercice,numérosiretsite,numérosirensite,...,libellérôle\n,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,0,10001410,10001410,M,71,Ostéopathe,L,"""Lib",indép,artis,...,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,NaN
1,0,10003887,10003887,MME,71,Ostéopathe,S,Salarié,26010004500012,NaN,...,\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,10005478,10005478,MME,71,Ostéopathe,L,"""Lib",indép,artis,...,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,NaN
3,0,10005684,10005684,MME,71,Ostéopathe,L,"""Lib",indép,artis,...,NaN,NaN,NaN,\n,NaN,NaN,NaN,NaN,NaN,NaN
4,0,19111228,19111228,MME,91,Orthophoniste,L,"""Lib",indép,artis,...,SA07,Cabinet individuel,FON-01,Titulaire de cabinet\n,NaN,NaN,NaN,NaN,NaN,NaN
